In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import time
import os
import sys
import multiprocessing as mp
import pymongo
from pubcrawler.article import Article
from epitator.annodoc import AnnoDoc
from epitator.annotier import AnnoTier
from epitator.geoname_annotator import GeonameAnnotator

from reporter import Reporter, MongoQueryReporter

In [3]:
import re

Find one article and count geonames

In [4]:
articles = pymongo.MongoClient("localhost", 27017).pmc.articles
geoname_annotator = GeonameAnnotator()

In [5]:
record = articles.find_one("5005014")

In [7]:
if len(record["extracted_text"]) > 100000:
    all_geospans = 0
    n_geospans = 0
    geospan_density = 0
else:
    article = AnnoDoc(record["extracted_text"])
    article.add_tier(geoname_annotator)
    article.tiers.update({"parentheticals": article.create_regex_tier("\(.*?\)")})
    all_geospans = article.tiers["geonames"]
    geospans = AnnoTier([span for span in geospans if span.metadata["geoname"].score > 0.13])
    parentheticals = article.tiers["parentheticals"]
    nonparen_geospans = geospans.subtract_overlaps(parentheticals)
    paren_geospans = geospans.subtract_overlaps(nonparen_geospans)
    
    n_all_geospans = len(all_geospans)
    n_geospans = len(geospans)
    n_nonparen_geospans = len(nonparen_geospans)
    n_paren_geospans = len(paren_geospans)
    geospan_density = n_geospans / len(record["extracted_text"])
    nonparen_geospan_density = n_nonparen_geospans / len(record["extracted_text"])
    paren_geospan_density = n_paren_geospans / len(record["extracted_text"])
articles.update_one(
            i,
            {
                "$set": {
                    "article_meta.all_geospans": all_geospans,
                    "article_meta.n_geospans": n_geospans,
                    "article_meta.geospan_density": geospan_density,
                }
            },
        )

NameError: name 'geospans' is not defined

In [7]:
article.tiers.keys()

dict_keys(['spacy.sentences', 'spacy.noun_chunks', 'spacy.tokens', 'spacy.nes', 'nes', 'tokens', 'ngrams', 'geonames'])

In [25]:
article.tiers.update({"parentheticals": article.create_regex_tier("\(.*?\)")})
parentheticals = article.tiers["parentheticals"]
nonparen_geospans = geospans.subtract_overlaps(parentheticals)
paren_geospans = geospans.subtract_overlaps(nonparen_geospans)

In [29]:
list(geospans.group_spans_by_containing_span(parentheticals))

[(AnnoSpan(4364-4395, University of Wisconsin (WIS)), []),
 (AnnoSpan(5579-5610, University of Wisconsin (WIS)), []),
 (AnnoSpan(8775-8784, Largo), []),
 (AnnoSpan(10154-10162, Jamshīdī), []),
 (AnnoSpan(10718-10733, Monroe), []),
 (AnnoSpan(11024-11038, Logan), []),
 (AnnoSpan(11093-11105, Manassas), []),
 (AnnoSpan(11142-11154, Manassas), []),
 (AnnoSpan(11233-11249, Grand Island), []),
 (AnnoSpan(11841-11857, Grand Island), []),
 (AnnoSpan(12284-12296, San Jose), []),
 (AnnoSpan(12335-12346, Ashland), []),
 (AnnoSpan(12948-12959, Raleigh), []),
 (AnnoSpan(12978-12991, San Diego County), []),
 (AnnoSpan(13142-13160, Franklin Lakes), []),
 (AnnoSpan(13481-13492, Raleigh), []),
 (AnnoSpan(15064-15076, Portland), []),
 (AnnoSpan(15336-15340, Isfahan), []),
 (AnnoSpan(15631-15644, Billerica), []),
 (AnnoSpan(15789-15791, Wisconsin), []),
 (AnnoSpan(16142-16150, Spearman), []),
 (AnnoSpan(16336-16344, Spearman), []),
 (AnnoSpan(17827-17839, Newfoundland and Labrador), []),
 (AnnoSpan(2766

In [151]:
match = matches[1]

In [152]:
match.groups(0)

('',)

In [149]:
match.end()

127

In [160]:
nonparen_tier = article.create_regex_tier("(<=\(.*?\)(.*?))", label = "non-parenthetical")
for span in nonparen_tier:
    print("text: {}".format(span.text))

In [98]:
nonparen_tier = article.create_regex_tier("(.*?)(?:\(.*?\)|$)", label = "non-parenthetical")
for span in nonparen_tier:
    print("text: {}".format(span.text))

text: Mid-substance rupture of the canine cranial cruciate ligament (CrCL)
text:  and associated stifle osteoarthritis (OA)
text:  is an important veterinary health problem [1]. Cruciate ligament rupture (CR)
text:  causes stifle joint instability and is not typically associated with contact trauma. Contralateral CR often develops, suggesting that the disease is a complex trait in which both genetic and environmental factors may increase risk of CR [1]. The dog is an important model for human anterior cruciate ligament (ACL)
text: Intrinsic healing of the CrCL/ACL is a research focus in veterinary and human medicine. Mesenchymal stem cells (also commonly known as mesenchymal stromal cells, MSCs)
text:  are multipotent adult progenitor cells that reside in virtually all tissues, including bone marrow (BM)
text: The early phase of the CrCL rupture mechanism, in which damage to CrCL matrix leads to partial CR in a stable stifle is not understood. Also, little is known about intrinsic heal

In [81]:
nonparen_tier = article.create_regex_tier("(?<!\().*?(?![\)])", label = "non-parenthetical")
for span in nonparen_tier:
    print("{}".format(span.text))

ligament
ligament
ligament
ligament
ligament
ligament
ligament
ligament
ligament
ligament
ligament
ligament
ligament
ligament
ligament
ligament
ligament
ligament
ligament
ligament


In [34]:
articles.update_one(
    i,
    {
        "$set": {
            "article_meta.all_geospans": all_geospans,
            "article_meta.n_geospans": n_geospans,
            "article_meta.geospan_density": geospan_density,
        }
    },
)

NameError: name 'i' is not defined

In [20]:
df

,n_spans,n_spans_over90,text_matches,any_matches
0,26,19,[study site],True
1,2,1,None,False
2,0,0,None,False
3,4,0,None,False
4,10,6,None,False
5,0,0,None,False
6,10,4,None,False
7,30,19,None,False
8,1,0,None,False
9,5,2,None,False


Sample specifically articles matching terms

In [71]:
with open("terms") as f:
    terms = [line.strip() for line in f.readlines()]

In [29]:
n = 250

In [32]:
terms_cursor = articles.find({ "text_matches": { "$in": terms } }, limit = n)
noterms_cursor = articles.find({ "text_matches": { "$not": {"$in": terms } } }, limit = n)
sample = list(terms_cursor) + list(noterms_cursor)

In [33]:
reporter = Reporter(5, len(sample))

rows = []
for idx, record in enumerate(sample):
    row = {}
    article = AnnoDoc(record["extracted_text"]).add_tier(geoname_annotator)
    geospans = article.tiers["geonames"]
    row["text_matches"] = record.get("text_matches")
    row["n_spans"] = len(geospans)
    row["n_spans_over90"] = sum([1 for span in geospans if span.metadata["geoname"].score > 0.9])
    rows.append(row)
    reporter.report(idx)

df = pd.DataFrame.from_records(rows)
df["any_matches"] = [False if row is None else True for row in df["text_matches"]]

Processed 500 articles (100.0%) in 31m57s; about 0m0s left.


In [56]:
df.to_csv(os.path.join("data", "geonames.csv"))

In [57]:
df.loc[df["any_matches"]].describe()

           n_spans  n_spans_over90
count   250.000000      250.000000
mean     52.204000       19.236000
std     168.625826       49.785608
min       0.000000        0.000000
25%      11.000000        3.000000
50%      24.000000        8.000000
75%      45.750000       20.000000
max    2448.000000      680.000000


In [58]:
df.loc[~df["any_matches"]].describe()

          n_spans  n_spans_over90
count  250.000000      250.000000
mean    12.508000        5.088000
std     17.606188        9.266267
min      0.000000        0.000000
25%      2.000000        0.000000
50%      7.000000        1.000000
75%     15.000000        5.000000
max    107.000000       59.000000
